In [1]:
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
import openpyxl as yxl
import pandas as pd
import os
import numpy as np

In [2]:
#Reference: https://strategyanalytics.medium.com/pandas-read-excel-removed-support-for-xlsx-files-426e4acfde89
df = pd.read_excel('CleanCountyProfile.xlsx', engine='openpyxl')


In [3]:
#O=increasing, 1=decreasing
data = df.reset_index(drop=True)
target1 = data["Land Class"]
target2 = data["GHG Rate"]
target_names = ["0", "1"]
data = data.drop(["GHG Rate", "Land Class", "County", "County.1", "Rural / Urban Status"], axis=1)
target1.name

'Land Class'

In [4]:
#Dropping historical data, taking only information that is indicative of a more recent period of time for the state (2015-2020)
data = data.drop(["Population, 1900", "Population, 1950", "Population, 1900", "Population, 2000", "Population, 2010", "% Change, 2000-10"], axis=1)
data = data.drop(["Population, 1990", "Estimated # Births, 2010 to 2019", "Estimated # Deaths, 2010 to 2019"], axis=1)
data = data.drop(["# Rural Municipalities, 2010", "# Urban Municipalities, 2010" ,"Median Age, 1990", "Median Age, 2000",
                 "Median Age, 2010", "% Households with  One or More People <18 years old, 2010", "Average # Persons in Household, 1990",
                 "Average # Persons in Household, 2000", "Average # Persons in Household, 2010", "New Housing Units Construction Permits, 2017",
                 "New Housing Units Construction Permits, 2018", "Median Household Income, 1998", "Median Household Income, 2008"], axis=1)


In [5]:
data = data.drop(["Per Capita Personal Income, 1998", "Per Capita Personal Income, 2008", "Gross Domestic Product (in $ Millions), 2010", 
                 "Gross Domestic Product (in $ Millions), 2015", "Total Poverty Rate, 1998", "Total Poverty Rate, 2008", "Poverty Rate for Children (<18 years old), 1998",
                 "Poverty Rate for Children (<18 years old), 2008", "Population Receiving Cash Assistance, June 2008", "Population Receiving Cash Assistance, June 2015", 
                  "Population Eligible for Medical Assistance (MA), June 2008", "Population Eligible for Medical Assistance (MA), June 2015", 
                  "Population Participating in SNAP, June 2008", "Population Participating in SNAP, June 2015", "% Change in Expenditures Per Student, 2014 to 2016",
                 "# to Receive Free & Reduced Lunch, 2016", "# to Receive Free & Reduced Lunch, 2017", "Graduation Rate (4-year Cohort), 2015-16"], axis=1)

In [6]:
data = data.drop(["Graduation Rate (4-year Cohort), 2016-17", "Average Unemployment Rate, 2017", 
                 "Average Unemployment Rate, 2018", "Average # Business Establishments, 2nd Qt. 2017", "Average # Business Establishments, 2nd Qt. 2018", 
                 "Average Employment, 2nd Qt. 2017", "Average Employment, 2nd Qt. 2018", "Avg. Time to Work (Minutes), 1990", "Avg. Time to Work (Minutes), 2000", 
                 "Avg. Time to Work (Minutes), 2010", "Total # Filings for Chapters 7, 11, 12, and 13, 2017", "Total # Filings for Chapters 7, 11, 12, and 13, 2018", 
                 " Live Births Per 1,000 Residents, 1988", " Live Births Per 1,000 Residents, 1998", " Live Births Per 1,000 Residents, 2008",
                 " Deaths Per 1,000 Residents, 1988", " Deaths Per 1,000 Residents, 1998", " Deaths Per 1,000 Residents, 2008", 
                 "# Infant Deaths per 1,000 Live Births, 2000 to 2004", "%  Without Health Insurance <65 Years Old, 2007",
                 "%  Without Health Insurance <65 Years Old, 2013", "Serious Crimes per 100,000 Residents, 2007", 
                 "Serious Crimes per 100,000 Residents, 2013", "Full-Time Police Officers per 1,000 Residents, 2007",
                 "Full-Time Police Officers per 1,000 Residents, 2013", "Voter Turnout, Nov. 2004", "Voter Turnout, Nov. 2008", 
                 "Voter Turnout, Nov. 2012", "# Farms, 2007", "# Farms, 2012", "Acres in Farmland, 2007", "Acres in Farmland, 2012",
                 "% Land In Farms, 2007", "% Land In Farms, 2012", "Average Size of Farms (acres), 2007", "Average Size of Farms (acres), 2012"], axis=1)

In [7]:
feature_names = data.columns
data

,Count of Democratic Voters,Count of Republican Voters,Count of No Affiliation Voters,"Population, 2019","% Change, 2010-19","Natural Population Change (Births Minus Deaths), 2010 to 2019",Net International Migration,Net Domestic Migration,Total Net Migration,"Land Area in Square Miles, 2010",...,"Farm Sales Under $10,000, 2017","Farm Sales $10,000-$49,999, 2017","Farm Sales $50,000+, 2017","Preserved Farms, April 2017","Acres of Preserved Farmland, April 2017",Population,Birth,Cancer,Death Rate,GDP
0,19264,39814,7990,103009,0.015798,189,544,926,1470,518.667519,...,0.458988,0.247818,0.293194,163,21434.02550,102811,17.9,460.2,743.3,3507548
1,528445,260457,91105,1216045,-0.005970,-6146,27900,-28057,-157,730.074498,...,0.681234,0.197943,0.120823,33,3458.34250,1218452,20.4,458.6,774.0,91296413
2,12585,25327,3157,64735,-0.061009,-1807,15,-2492,-2477,653.202617,...,0.567365,0.272455,0.160180,5,486.04900,65263,17.1,427.7,798.8,2322321
3,51464,46144,9440,163929,-0.038759,-4294,161,-2292,-2131,434.712442,...,0.678630,0.185971,0.135400,27,2932.56800,164742,18.7,467.5,836.0,7076904
4,6548,23043,2003,47888,-0.037659,-844,51,-1058,-1007,1012.296348,...,0.477998,0.235548,0.286454,17,3785.58625,48176,19.4,434.6,733.7,1448401
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,7794,14566,2317,39191,-0.062753,-982,87,-1705,-1618,884.134793,...,0.634956,0.205752,0.159292,2,309.83900,39498,21.1,437.4,767.7,1766395
60,59728,64826,13138,206865,-0.004595,-5086,805,3532,4337,856.988913,...,0.639773,0.273864,0.086364,34,5393.52390,207346,19.1,477.1,817.5,14054804
61,98169,117886,19054,348899,-0.044555,-12702,626,-3780,-3154,1027.554384,...,0.586897,0.256597,0.156506,95,12616.65700,350611,16.3,447.1,798.1,13532650
62,5050,10537,1295,26794,-0.052412,-221,62,-1322,-1260,397.323492,...,0.612195,0.229268,0.158537,10,1569.47000,27046,17.8,429.5,954.6,2207240


In [8]:
for feature in feature_names: 
    print(feature)

Count of Democratic Voters
Count of Republican Voters
Count of No Affiliation Voters
Population, 2019
% Change, 2010-19
Natural Population Change (Births Minus Deaths), 2010 to 2019
Net International Migration
Net Domestic Migration
Total Net Migration
Land Area in Square Miles, 2010
Population per Square Mile
# Municipalities, 2018
With Population fewer Than 2,500
With Population of 2,500 to 4,999
With Population 5,000 to 9,999
With Population 10,000+
# Persons <18, 2018
% Population <18 Years Old
# Persons 18 to 64, 2018
% Population 18 to 64 Years Old
# Persons 65+, 2018
% Population 65+ Years Old
Median Age, 2018
# Persons White Only, 2018
% Population White Only
# Persons Black/African American Only, 2018
% Population Black/African American Only
# Persons Asian Only, 2018
% Population Asian Only
# Persons of Other Races and Two or More Races, 2018
% Population Other Races and Two or More Races
# Person of Hispanic/Latino Origin (All Races), 2018
% Population Hispanic/Latino Origin

In [9]:
voting_df = data[["Count of Democratic Voters", "Count of Republican Voters", "Count of No Affiliation Voters", "Voter Turnout, Nov. 2016"]]

general_population_df = data[["Population, 2019", "% Change, 2010-19",
                              "Natural Population Change (Births Minus Deaths), 2010 to 2019",
                              "Net International Migration",
                              "Net Domestic Migration",
                              "Total Net Migration"]]
municipalities_df = data[["# Municipalities, 2018", "With Population fewer Than 2,500", "With Population of 2,500 to 4,999",
                          "With Population 5,000 to 9,999", "With Population 10,000+", 
                         " # of Municipalities, 2018", "% With Municipal Comprehensive Plans", "% With Municipal Planning Commissions",
                          "% With Municipal and/or County Zoning Ordinances"]]

age_df = data[["# Persons <18, 2018","% Population <18 Years Old", "# Persons 18 to 64, 2018", "% Population 18 to 64 Years Old",
              "# Persons 65+, 2018",
              "% Population 65+ Years Old",
              "Median Age, 2018"]]


race_ethnicity_df = data[["# Persons White Only, 2018", "% Population White Only", "# Persons Black/African American Only, 2018",
                     "% Population Black/African American Only", "# Persons Asian Only, 2018", "% Population Asian Only",
                     "# Persons of Other Races and Two or More Races, 2018", "% Population Other Races and Two or More Races",
                     "# Person of Hispanic/Latino Origin (All Races), 2018", "% Population Hispanic/Latino Origin",
                     "% Native Born Residents, 2018", "% Foreign Born Residents, 2018"]]


households_and_familiesdf = data[["# Households, 2018", "# Families, 2018", "Total # Households, 2018",
                                 "% Married Couples With Own Children (<18)", "% Married Couples With No Children",
                                 "% Single Parents (Male/Female, No Spouse, Children <18)", "% Single Person Households",
                                 "% Other Types of Households", "% Households with One or More People <18 years old, 2000",
                                 "% Households with  One or More People <18 years old, 2018", 
                                  "Average # Persons in Household, 2018"]]

housing_by_unitdf = data[["# Housing Units, 2018", "% Change 2010 to 2018", "# Vacant Housing Units, 2018", "% Housing Units Vacant",
                         "% Vacant Units that are Seasonal, Recreational, or Occasional Use", 
                         "% Vacant Units for Rent/Sale ", "% Units Vacant for Other Reasons", "Total # Housing Units, 2018",
                         "% Single Family Home (Detached 1 Unit)", "% Duplex/Townhouse/Row Home (Attached 1 Unit)",
                         "% Small Apartment Type Building (Less than 9 Units)", "% Large Apartment Type Building (10 or More Units)",
                         "% Mobile Home & Other Types of Units", "# Occupied Housing Units, 2018", "% Homeownership Rate (Owner-Occupied Units)",
                         "% Renters (Renter-Occupied Units)", "New Housing Units Construction Permits, 2019"]]

housing_affordabilitydf = data[["Median Housing Value, 2018",
                                "Average Housing Value, 2018",
                                "Median Gross Monthly Rent, 2018",
                                "Average Gross Monthly Rent, 2018",
                                "Homeowners with Mortgage Whose Income/Housing Cost Ratios were Calculated, 2018",
                                "% Homeowners Paying Less than 30% of Income for Housing",
                                "% Homeowners Paying 30% to 49% of Income for Housing",
                                "% Homeowners Paying 50%+ of Income for Housing",
                                "# Renters For Whom Income/Rent Ratios were Calculated, 2018",
                                "% Renters Paying Less than 30% of Income for Housing",
                                "% Renters Paying 30% to 49% of Income for Housing",
                                "% Renters Paying 50%+ of Income for Housing"]]


household_incomedf = data[["# Households, 2018.1",
                            "% Households with Internet Access",
                            "% Households with No Internet Access",
                            "Median Household Income, 2018",
                            "# Households with Income, 2018",
                            "<$25,000 Household Income",
                            "$25,000 to $49,999 Household Income",
                            "$50,000 to $99,999 Household Income",
                            "$100,000 to $149,999 Household Income",
                            "$150,000+ Household Income",
                            "Average Household Income, 2018",
                            "# Households with Income, 2018.1",
                            "% Households with Income from Wages & Salaries",
                            "% Households with Income from Social Security ", 
                            "% Households with Income from Supplemental Security Income (SSI)",
                            "% Households with Income from Public Assistance",
                            "% Households with Income from Retirement "]] 



In [10]:
income_and_GDPdf = data[["Per Capita Personal Income, 2018",
                            "Total Personal Income ($1,000), 2018",
                            "% Income from Wages & Salaries",
                            "% Income from Dividends, Interest, & Rent",
                            "% Income from Transfer Payments",
                            "Gross Domestic Product (in $ Millions), 2018",
                            "Gross Domestic Product Per Capita, 2018",
                            "Total Poverty Rate, 2018",
                            "Poverty Rate for Children (<18 years old), 2018",
                            "Population Receiving Cash Assistance, June 2019 (Est.)"]]


public_programsdf = data[["Population Eligible for Medical Assistance (MA), June 2019",
                            "Population Participating in SNAP, June 2019",
                            "Total Number Enrolled in PACE/PACENET, 2018",
                            "% Population 65+ Enrolled in PACE/PACENET, 2018"]]


disabilitydf = data[["# Non-Institutionalized Persons 18 to 64 with Disability, 2018",
                        "% Population 18 to 64 with Disability",
                        "# Non-Institutionalized Persons 65+ with Disability, 2018",
                        "% Population 65+ with Disability"]]




In [11]:
educationdf = data[["# Persons, 25 Years Old & Older, 2018",
                    "No High School Diploma",
                    "High School Diploma or Equivalency",
                    "Some College, No Degree",
                    "Associate's Degree",
                    "Bachelor's Degree or Higher",
                    "Total # Students Enrolled in Grades K-12, 2018",
                    "% Enrolled in Public Schools",
                    "% Enrolled in Private Schools",
                    "# School Districts",
                    "# Students (Average Daily Membership), 2017-18",
                    "% Change, 2008-2018",
                    "Projected Enrollment, 2023-2024",
                    "Projected Enrollment, 2028-2029",
                    "Total School Revenues  ($1,000), 2017-18",
                    "% Revenues from Local Sources",
                    "% Revenues from State Sources",
                    "% Revenues from Federal & Other Sources",
                    "Total Expenditures Per Student, 2017-18",
                    "# to Receive Free & Reduced Lunch, 2018",
                    "Graduation Rate (4-year Cohort), 2017-18",
                    "# Degree Granting Institutions (Colleges & Universities), 2018",
                    "# Non-Degree Granting Institutions (Trade & Technical schools), 2018"]]

employmentdf = data[["Total # Employed Persons, 2018",
                        "% Manufacturing",
                        "% Wholesale & Retail Trade",
                        "% Mining, Construction, Utilities, & Transportation & Warehousing",
                        "% Information, Finance & Insurance, & Real Est. & Rental & Leasing",
                        "% Education Serv. & Professional, Scientific, & Technical Services",
                        "% Health Care & Social Assistance",
                        "% Accommodation & Food Services",
                        "% Other Sectors including Public Administration",
                        "# Employed Persons, 2018",
                        "% Management, Professional, and Related",
                        "% Service",
                        "% Sales and Office",
                        "% Farming, Fishing, and Forestry",
                        "% Construction, Extraction, Maintenance, and Repair",
                        "% Production, Transportation, and Material Moving",
                        "Average Unemployment Rate, 2019",
                        "Average # Business Establishments, 2nd Qt. 2019",
                        "Average Employment, 2nd Qt. 2019",
                        "Average # Employed Persons in Workforce, 2018",
                        "% of Employed Persons <25 Years Old",
                        "% Employed Persons 25 to 54 Years Old",
                        "% Employed Persons 55+ Years Old",
                        "# Employed Persons 16 Years Old & Older, 2018",
                        "% Who Worked in County of Residence",
                        "% Who Worked Outside County of Residence, but Within PA",
                        "% Who Worked Outside Pennsylvania",
                        "Avg. Time to Work (Minutes), 2018",
                        "Total # Filings for Chapters 7, 11, 12, and 13, 2019"]]




In [12]:
birthdf = data[[" Live Births Per 1,000 Residents, 2018",
                " Deaths Per 1,000 Residents (Preliminary), 2018",
                "# Reported Pregnancies, 2015-17",
                "% Resulting in Live Births",
                "% Ending in Fetal Deaths",
                "% Ending in Induced Abortions",
                "% Reported Pregnancies of Women <18 Years Old",
                "% Born With Low Birth Weight (Under 2,500 Grams), 2016-2018",
                "% Born to Unmarried Mothers, 2016-2018",
                "% Born to Mothers Who Received Medicaid, 2016-2018",
                "% Born to Mothers Who Did Not Receive Prenatal Care in 1st Trimester, 2016-2018"]]

deathdf = data[["Resident Deaths, 2015-2017",
                "Death Rate Per 1,000 Residents, All Causes",
                "Heart Disease Death Rate per 1,000 Residents",
                "Cancer Death Rate per 1,000 Residents",
                "Drug Overdose Death Rate per 1,000 Residents",
                "Suicide Death Rate per 1,000 Residents"]]

medical_caredf = data[[" General Acute Care Hospitals, 2018",
                        "Hospital Beds Set Up & Staffed, 2018",
                        " Beds Set Up & Staffed per 1,000 Residents",
                        "# Nursing Homes, 2017-18",
                        "Total # Licensed/Approved Nursing Home Beds, 2017-18",
                        "Total # Licensed/Approved Nursing Home Beds per 1,000 Residents, 2017-18",
                        "# Primary Care Physicians (MD/DO), 2017",
                        "# Primary Care Physicians per 100,000 Residents, 2017",
                        "# Nurse Practitioners, 2017",
                        "# Nurse Practitioners per 100,000 Residents, 2017",
                        "# Dentists, 2017",
                        "# Dentists per 100,000 Residents, 2017",
                        "%  Without Health Insurance <65 Years Old, 2017"]]

policing_df = data[["Serious Crimes per 100,000 Residents, 2017",
                    "Full-Time Police Officers per 1,000 Residents, 2017"]]

highway_and_drivingdf = data[["Total Miles of Highway, 2018",
                            "% PennDOT and Other State & Federal Highways",
                            "% Local (Municipal) Highways",
                            "# Licensed Drivers, January, 2019",
                            "% Change in Licensed Drivers, 2009-2019",
                            " In-State Registered Vehicles, 2018",
                            " In-State Registered Vehicles Per 1,000 Residents",
                            "Change in In-State Registered Vehicles, 2008-2018"]]

farmdf = data[["# Farms, 2017",
                "Acres in Farmland, 2017",
                "% Land In Farms, 2017",
                "Average Size of Farms (acres), 2017",
                "Total Market Value of Ag. Products Sold ($1,000), 2017",
                "Average Market Value of Products Sold Per Farm, 2017",
                "Average Market Value of Products Sold Per Acre, 2017",
                "Farm Sales Under $10,000, 2017",
                "Farm Sales $10,000-$49,999, 2017",
                "Farm Sales $50,000+, 2017",
                "Preserved Farms, April 2017",
                "Acres of Preserved Farmland, April 2017"]]

randomdf = data[["Population",
                    "Birth",
                    "Cancer",
                    "Death Rate",
                    "GDP"]]




In [13]:
total_dfs = [{'name': "Voter Data" , 'data': voting_df},
            {'name': "General Population Data" , 'data': general_population_df},
            {'name': "Municipal Data" , 'data': municipalities_df},
            {'name': "Age Data" , 'data': age_df},
            {'name': "Race/Ethnicity Data" , 'data': race_ethnicity_df},
            {'name': "Family Unit Data" , 'data': households_and_familiesdf},
            {'name': "Housing Unit Data" , 'data': housing_by_unitdf},
            {'name': "Housing Affordability Data" , 'data': housing_affordabilitydf},
            {'name': "Household Income Data" , 'data': household_incomedf},
            {'name': "Economic Data" , 'data': income_and_GDPdf},
            {'name': "Public Program Data" , 'data': public_programsdf},
            {'name': "Disability Data" , 'data': disabilitydf},
            {'name': "Education Data" , 'data': educationdf},
            {'name': "Employment Data" , 'data': employmentdf},
            {'name': "Birth Data" , 'data': birthdf},
            {'name': "Death Data" , 'data': deathdf},
            {'name': "Medical Care Data" , 'data': medical_caredf},
            {'name': "Policing Data" , 'data': policing_df},
            {'name': "Highway and Driving Data" , 'data': highway_and_drivingdf},
            {'name': "Farm Data" , 'data': farmdf},
            {'name': "Randomly Associated Test Data" , 'data': randomdf}]



In [14]:
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
# Create a StandardScater model and fit it to the training data

def runRFM(dataframe, target_col):

    feature_names = dataframe.columns
    X_train, X_test, y_train, y_test = train_test_split(dataframe, target_col, random_state=42)
    X_scaler = StandardScaler().fit(X_train)
    X_train_scaled = X_scaler.transform(X_train)
    X_test_scaled = X_scaler.transform(X_test)
    
    rf = RandomForestClassifier(n_estimators=200)
    rf = rf.fit(X_train_scaled, y_train)
    rf_feature_importances = sorted(zip(rf.feature_importances_, feature_names), reverse=True)
    accuracy = rf.score(X_test_scaled, y_test)
    output = {'Importances': rf_feature_importances, 'Accuracy': accuracy}
    
    return output

In [15]:
def RFMiterator1(alldatasets, total_iterations, target_col):
        
    for iteration in range(total_iterations):
        
        land_accuracies = [] 
        dataset_names = []
    
        for i in range(len(alldatasets)):

            rfm_output1 = runRFM(alldatasets[i]['data'], target_col)

#             print("\nBegin Calculation for " + alldatasets[i]['name'])
#             print('Overall Model Accuracy: ', rfm_output1['Accuracy'])
#             print('Overall Importances: ')
#             for importance in rfm_output1['Importances']:
#                 print(importance)
            land_accuracies.append(rfm_output1['Accuracy'])
            dataset_names.append(alldatasets[i]['name'])
#             print('End Calculation\n')
        
        if(iteration == 0):
        
            accuracies = pd.DataFrame({"Dataset Names": dataset_names, "Land Accuracies 0": land_accuracies})
            
        accuracies['Land Accuracies ' + str(iteration)] = land_accuracies

    
    print('End Calculation\n') 
    
    return accuracies
    

In [16]:
def RFMiterator2(alldatasets, total_iterations, target_col):
        
    for iteration in range(total_iterations):
        
        ghg_accuracies = [] 
        dataset_names = []
    
        for i in range(len(alldatasets)):

            rfm_output2 = runRFM(alldatasets[i]['data'], target_col)

#             print("\nBegin Calculation for " + alldatasets[i]['name'])
#             print('Overall Model Accuracy: ', rfm_output1['Accuracy'])
#             print('Overall Importances: ')
#             for importance in rfm_output1['Importances']:
#                 print(importance)
            ghg_accuracies.append(rfm_output2['Accuracy'])
            dataset_names.append(alldatasets[i]['name'])
#             print('End Calculation\n')
        
        if(iteration == 0):
        
            accuracies = pd.DataFrame({"Dataset Names": dataset_names, "GHG Accuracies 0": ghg_accuracies})
            
        accuracies['GHG Accuracies ' + str(iteration)] = ghg_accuracies

    
    print('End Calculation\n') 
    
    return accuracies
    

In [17]:
statisticalset1 = RFMiterator1(total_dfs, 50, target1)

End Calculation



In [18]:
statisticalset1

,Dataset Names,Land Accuracies 0,Land Accuracies 1,Land Accuracies 2,Land Accuracies 3,Land Accuracies 4,Land Accuracies 5,Land Accuracies 6,Land Accuracies 7,Land Accuracies 8,...,Land Accuracies 40,Land Accuracies 41,Land Accuracies 42,Land Accuracies 43,Land Accuracies 44,Land Accuracies 45,Land Accuracies 46,Land Accuracies 47,Land Accuracies 48,Land Accuracies 49
0,Voter Data,0.5000,0.5000,0.5000,0.5000,0.5000,0.5000,0.5000,0.5000,0.5000,...,0.5000,0.5000,0.5000,0.5000,0.5000,0.5000,0.5000,0.5000,0.5000,0.5000
1,General Population Data,0.6875,0.5625,0.6250,0.6875,0.6875,0.6250,0.6875,0.6875,0.6875,...,0.6250,0.6250,0.5625,0.6875,0.6875,0.6250,0.6875,0.6250,0.6250,0.6250
2,Municipal Data,0.6875,0.6875,0.6875,0.6875,0.6875,0.6875,0.6875,0.6875,0.6875,...,0.6875,0.6875,0.6875,0.6875,0.6875,0.6875,0.6875,0.6875,0.6875,0.6875
3,Age Data,0.6250,0.6875,0.6875,0.6875,0.6875,0.7500,0.6250,0.6875,0.6875,...,0.6875,0.6875,0.6875,0.7500,0.6875,0.7500,0.6875,0.7500,0.6875,0.6875
4,Race/Ethnicity Data,0.6875,0.6875,0.7500,0.6875,0.6875,0.7500,0.7500,0.6875,0.7500,...,0.6875,0.6875,0.6875,0.6875,0.6250,0.7500,0.7500,0.6875,0.7500,0.7500
5,Family Unit Data,0.6875,0.6250,0.6250,0.6875,0.6875,0.6875,0.6875,0.6875,0.6250,...,0.6250,0.6875,0.6250,0.6875,0.6875,0.6875,0.6875,0.5625,0.6875,0.6875
6,Housing Unit Data,0.6875,0.6250,0.6250,0.6250,0.6250,0.6875,0.6875,0.6875,0.6250,...,0.6875,0.6875,0.6875,0.6875,0.6875,0.6250,0.6875,0.7500,0.6250,0.6875
7,Housing Affordability Data,0.7500,0.6875,0.7500,0.7500,0.7500,0.7500,0.7500,0.6875,0.7500,...,0.7500,0.7500,0.6875,0.7500,0.7500,0.7500,0.6875,0.7500,0.7500,0.7500
8,Household Income Data,0.5625,0.5625,0.6250,0.5625,0.6250,0.5625,0.6250,0.5625,0.5625,...,0.6250,0.6250,0.5625,0.5625,0.5625,0.6250,0.5625,0.6250,0.5625,0.6250
9,Economic Data,0.6250,0.6250,0.6250,0.6875,0.6250,0.6875,0.6875,0.6250,0.6875,...,0.6875,0.6875,0.6875,0.6250,0.6875,0.6875,0.6875,0.6875,0.6875,0.6875


In [19]:
statisticalset1['Average'] = statisticalset1.mean('columns')
statisticalset1['Max'] = statisticalset1.max('columns')
statisticalset1['Min'] = statisticalset1.min('columns')
statisticalset1['Standard Deviation'] = statisticalset1.std('columns')
statisticalset1['Variance'] = statisticalset1.var('columns')

statisticalset1[['Dataset Names','Average', 'Standard Deviation', 'Variance', 'Max', 'Min']]

,Dataset Names,Average,Standard Deviation,Variance,Max,Min
0,Voter Data,0.50000,0.000000,0.004630,0.5000,0.5000
1,General Population Data,0.64250,0.046051,0.008654,0.6875,0.5625
2,Municipal Data,0.68625,0.012020,0.008532,0.6875,0.6250
3,Age Data,0.70000,0.038835,0.009563,0.7500,0.6250
4,Race/Ethnicity Data,0.71625,0.037607,0.009889,0.7500,0.6250
5,Family Unit Data,0.65875,0.037761,0.008511,0.6875,0.5625
6,Housing Unit Data,0.66125,0.053792,0.009695,0.8125,0.5625
7,Housing Affordability Data,0.73375,0.027723,0.009970,0.7500,0.6875
8,Household Income Data,0.59375,0.037761,0.007148,0.6875,0.5625
9,Economic Data,0.66625,0.029734,0.008361,0.6875,0.6250


In [20]:
statisticalset2 = RFMiterator2(total_dfs, 50, target2)

End Calculation



In [21]:
statisticalset2

,Dataset Names,GHG Accuracies 0,GHG Accuracies 1,GHG Accuracies 2,GHG Accuracies 3,GHG Accuracies 4,GHG Accuracies 5,GHG Accuracies 6,GHG Accuracies 7,GHG Accuracies 8,...,GHG Accuracies 40,GHG Accuracies 41,GHG Accuracies 42,GHG Accuracies 43,GHG Accuracies 44,GHG Accuracies 45,GHG Accuracies 46,GHG Accuracies 47,GHG Accuracies 48,GHG Accuracies 49
0,Voter Data,0.5000,0.5625,0.5000,0.5000,0.4375,0.5000,0.5000,0.5000,0.5000,...,0.5000,0.5000,0.5000,0.5000,0.5000,0.5000,0.5000,0.5000,0.5000,0.5000
1,General Population Data,0.3750,0.3750,0.5625,0.3750,0.5625,0.5000,0.4375,0.5000,0.5000,...,0.4375,0.5000,0.4375,0.4375,0.5000,0.3750,0.5000,0.3125,0.5000,0.5000
2,Municipal Data,0.5625,0.5000,0.5000,0.4375,0.4375,0.4375,0.5625,0.5000,0.5000,...,0.5000,0.5000,0.5000,0.5000,0.5000,0.5000,0.5000,0.5000,0.5000,0.5000
3,Age Data,0.6250,0.6250,0.5625,0.5000,0.6250,0.6250,0.5000,0.5625,0.6250,...,0.6250,0.5000,0.6250,0.5625,0.6250,0.5625,0.6250,0.6250,0.5625,0.5625
4,Race/Ethnicity Data,0.5625,0.4375,0.5000,0.4375,0.3750,0.4375,0.4375,0.3750,0.5000,...,0.4375,0.3750,0.5000,0.5000,0.5000,0.5000,0.3750,0.4375,0.4375,0.4375
5,Family Unit Data,0.5625,0.6875,0.6875,0.6250,0.6875,0.6250,0.6875,0.6875,0.6250,...,0.6875,0.6875,0.6875,0.6250,0.5625,0.6875,0.6875,0.5625,0.5625,0.6875
6,Housing Unit Data,0.5000,0.5000,0.4375,0.4375,0.4375,0.5000,0.4375,0.4375,0.5000,...,0.4375,0.4375,0.3750,0.4375,0.5000,0.3750,0.3125,0.3750,0.4375,0.4375
7,Housing Affordability Data,0.5625,0.5625,0.6250,0.5000,0.5625,0.5625,0.6250,0.5625,0.6250,...,0.5000,0.5000,0.6250,0.5625,0.5000,0.5625,0.5625,0.6250,0.5625,0.5000
8,Household Income Data,0.5000,0.4375,0.3750,0.5000,0.3750,0.3750,0.5000,0.3750,0.4375,...,0.3750,0.3750,0.4375,0.4375,0.5000,0.4375,0.4375,0.3750,0.4375,0.4375
9,Economic Data,0.4375,0.3750,0.3750,0.4375,0.4375,0.5625,0.3750,0.5625,0.4375,...,0.3125,0.4375,0.5000,0.4375,0.4375,0.4375,0.4375,0.3750,0.3750,0.4375


In [22]:
statisticalset2['Average'] = statisticalset2.mean('columns')
statisticalset2['Max'] = statisticalset2.max('columns')
statisticalset2['Min'] = statisticalset2.min('columns')
statisticalset2['Standard Deviation'] = statisticalset1.std('columns')
statisticalset2['Variance'] = statisticalset2.var('columns')

statisticalset2[['Dataset Names','Average', 'Standard Deviation', 'Variance', 'Max', 'Min']]

,Dataset Names,Average,Standard Deviation,Variance,Max,Min
0,Voter Data,0.50125,0.094023,0.003585,0.5625,0.4375
1,General Population Data,0.48250,0.124626,0.007279,0.6250,0.3125
2,Municipal Data,0.49250,0.128035,0.004034,0.5625,0.4375
3,Age Data,0.58625,0.133182,0.006335,0.6875,0.5000
4,Race/Ethnicity Data,0.45750,0.135761,0.004976,0.5625,0.3750
5,Family Unit Data,0.64625,0.125468,0.007570,0.6875,0.5000
6,Housing Unit Data,0.44125,0.130354,0.003833,0.5000,0.3125
7,Housing Affordability Data,0.56625,0.137677,0.006556,0.6875,0.4375
8,Household Income Data,0.43125,0.114361,0.004996,0.5625,0.3125
9,Economic Data,0.43375,0.125649,0.005358,0.5625,0.3125


In [23]:
#https://treyhunner.com/2016/04/how-to-loop-with-indexes-in-python/
# land_accuracies = [] 
# ghg_rate_accuracies = [] 
# dataset_names = []


# for i in range(len(total_dfs)):

#     rfm_output1 = runRFM(total_dfs[i]['data'], target1)

#     print("\nBegin Calculation for " + total_dfs[i]['name'])
#     print('Overall Model Accuracy: ', rfm_output1['Accuracy'])
#     print('Overall Importances: ')
#     for importance in rfm_output1['Importances']:
#         print(importance)
#     land_accuracies.append(rfm_output1['Accuracy'])
#     dataset_names.append(total_dfs[i]['name'])
#     print('End Calculation\n')

    

In [24]:
#https://treyhunner.com/2016/04/how-to-loop-with-indexes-in-python/
# for i in range(len(total_dfs)):
    
#     rfm_output2 = runRFM(total_dfs[i]['data'], target2)
    
#     print("\nBegin Calculation for " + total_dfs[i]['name'])
#     print('Overall Model Accuracy: ', rfm_output2['Accuracy'])
#     print('Overall Importances: ')
#     for importance in rfm_output2['Importances']:
#         print(importance)
#     ghg_rate_accuracies.append(rfm_output2['Accuracy'])
#     print('End Calculation\n')

In [25]:
avg_accuracy_df = pd.DataFrame({'Dataset Names': statisticalset1['Dataset Names'], 'Land Classifier Avg. Accuracy': statisticalset1['Average'],
                               'CO2 Rate Avg. Accuracy': statisticalset2['Average']})

avg_accuracy_df

,Dataset Names,Land Classifier Avg. Accuracy,CO2 Rate Avg. Accuracy
0,Voter Data,0.50000,0.50125
1,General Population Data,0.64250,0.48250
2,Municipal Data,0.68625,0.49250
3,Age Data,0.70000,0.58625
4,Race/Ethnicity Data,0.71625,0.45750
5,Family Unit Data,0.65875,0.64625
6,Housing Unit Data,0.66125,0.44125
7,Housing Affordability Data,0.73375,0.56625
8,Household Income Data,0.59375,0.43125
9,Economic Data,0.66625,0.43375


In [26]:
# avg_accuracy_df.to_csv('RFM_avg_accuracies.csv', index=False)

In [27]:
#Support Vector Machine
#Reference: https://stackoverflow.com/questions/39662398/scikit-learn-output-metrics-classification-report-into-csv-tab-delimited-format
from sklearn.svm import SVC 
from sklearn.metrics import classification_report

def runSVC(dataframe, target_col):

    feature_names = dataframe.columns
    X_train, X_test, y_train, y_test = train_test_split(dataframe, target_col, random_state=42)
    X_scaler = StandardScaler().fit(X_train)
    X_train_scaled = X_scaler.transform(X_train)
    X_test_scaled = X_scaler.transform(X_test)

    
    svc = SVC(kernel='rbf', gamma=1, C=10)
    svc = svc.fit(X_train_scaled, y_train)
    svc_predictions = svc.predict(X_test_scaled) 
    output = pd.DataFrame(classification_report(y_test, svc_predictions, target_names = ["0", "1"], output_dict=True)).transpose()
#     print("Predictions:\n", svc_predictions)
#     print("X_train_Scaled\n", X_train_scaled)
#     print("X_test_scaled\n", X_test_scaled)
#     print("Y_train\n", y_train)
#     print("Y_test\n", y_test)
    
    return output


In [28]:
def SVCiterator1(alldatasets, total_iterations, target_col):
    
    classifier_name = f"SVM {target_col.name.upper()} F1-Score"
    f1_score_dict = {"Dataset Names": [], classifier_name: []}
        
    for iteration in range(total_iterations):
        
        for i in range(len(alldatasets)):
            
            df_name = total_dfs[i]['name']
            
            print("\nBegin Calculation for " + df_name)
            svc_output = runSVC(alldatasets[i]['data'], target_col)
            f1_score = svc_output.loc['accuracy','f1-score']
            f1_score_dict["Dataset Names"].append(df_name)
            f1_score_dict[classifier_name].append(f1_score)
            
            print(svc_output)
    
    print('End Calculation\n')
#     print(f1_score_dict)
    return f1_score_dict
    

In [29]:
landoutput = SVCiterator1(total_dfs, 1, target1)
alldatasets_SVC_landclass = pd.DataFrame(landoutput)
alldatasets_SVC_landclass


Begin Calculation for Voter Data
              precision    recall  f1-score  support
0              0.250000  0.200000  0.222222   5.0000
1              0.666667  0.727273  0.695652  11.0000
accuracy       0.562500  0.562500  0.562500   0.5625
macro avg      0.458333  0.463636  0.458937  16.0000
weighted avg   0.536458  0.562500  0.547705  16.0000

Begin Calculation for General Population Data
              precision    recall  f1-score  support
0              0.166667  0.200000  0.181818   5.0000
1              0.600000  0.545455  0.571429  11.0000
accuracy       0.437500  0.437500  0.437500   0.4375
macro avg      0.383333  0.372727  0.376623  16.0000
weighted avg   0.464583  0.437500  0.449675  16.0000

Begin Calculation for Municipal Data
              precision    recall  f1-score  support
0              0.500000  0.400000  0.444444   5.0000
1              0.750000  0.818182  0.782609  11.0000
accuracy       0.687500  0.687500  0.687500   0.6875
macro avg      0.625000  0.609091

C:\Users\ahop9\anaconda3\envs\PythonAdv\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ahop9\anaconda3\envs\PythonAdv\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ahop9\anaconda3\envs\PythonAdv\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\a

              precision  recall  f1-score  support
0              0.000000  0.0000  0.000000   5.0000
1              0.687500  1.0000  0.814815  11.0000
accuracy       0.687500  0.6875  0.687500   0.6875
macro avg      0.343750  0.5000  0.407407  16.0000
weighted avg   0.472656  0.6875  0.560185  16.0000

Begin Calculation for Economic Data
              precision    recall  f1-score  support
0              0.750000  0.600000  0.666667   5.0000
1              0.833333  0.909091  0.869565  11.0000
accuracy       0.812500  0.812500  0.812500   0.8125
macro avg      0.791667  0.754545  0.768116  16.0000
weighted avg   0.807292  0.812500  0.806159  16.0000

Begin Calculation for Public Program Data
              precision    recall  f1-score  support
0              0.500000  0.400000  0.444444   5.0000
1              0.750000  0.818182  0.782609  11.0000
accuracy       0.687500  0.687500  0.687500   0.6875
macro avg      0.625000  0.609091  0.613527  16.0000
weighted avg   0.671875  0.6875

C:\Users\ahop9\anaconda3\envs\PythonAdv\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ahop9\anaconda3\envs\PythonAdv\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ahop9\anaconda3\envs\PythonAdv\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,Dataset Names,SVM LAND CLASS F1-Score
0,Voter Data,0.5625
1,General Population Data,0.4375
2,Municipal Data,0.6875
3,Age Data,0.8125
4,Race/Ethnicity Data,0.5625
5,Family Unit Data,0.7500
6,Housing Unit Data,0.5625
7,Housing Affordability Data,0.7500
8,Household Income Data,0.6875
9,Economic Data,0.8125


In [30]:
rateoutput = SVCiterator1(total_dfs, 1, target2)
alldatasets_SVC_rateclass = pd.DataFrame(rateoutput)
alldatasets_SVC_rateclass


Begin Calculation for Voter Data
              precision    recall  f1-score  support
0              0.600000  0.300000  0.400000  10.0000
1              0.363636  0.666667  0.470588   6.0000
accuracy       0.437500  0.437500  0.437500   0.4375
macro avg      0.481818  0.483333  0.435294  16.0000
weighted avg   0.511364  0.437500  0.426471  16.0000

Begin Calculation for General Population Data
              precision  recall  f1-score  support
0              0.400000  0.2000  0.266667  10.0000
1              0.272727  0.5000  0.352941   6.0000
accuracy       0.312500  0.3125  0.312500   0.3125
macro avg      0.336364  0.3500  0.309804  16.0000
weighted avg   0.352273  0.3125  0.299020  16.0000

Begin Calculation for Municipal Data
              precision    recall  f1-score  support
0              0.500000  0.100000  0.166667   10.000
1              0.357143  0.833333  0.500000    6.000
accuracy       0.375000  0.375000  0.375000    0.375
macro avg      0.428571  0.466667  0.333333  

C:\Users\ahop9\anaconda3\envs\PythonAdv\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ahop9\anaconda3\envs\PythonAdv\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ahop9\anaconda3\envs\PythonAdv\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\a

              precision  recall  f1-score  support
0              0.000000   0.000  0.000000   10.000
1              0.375000   1.000  0.545455    6.000
accuracy       0.375000   0.375  0.375000    0.375
macro avg      0.187500   0.500  0.272727   16.000
weighted avg   0.140625   0.375  0.204545   16.000

Begin Calculation for Death Data
              precision  recall  f1-score  support
0              0.400000  0.2000  0.266667  10.0000
1              0.272727  0.5000  0.352941   6.0000
accuracy       0.312500  0.3125  0.312500   0.3125
macro avg      0.336364  0.3500  0.309804  16.0000
weighted avg   0.352273  0.3125  0.299020  16.0000

Begin Calculation for Medical Care Data
              precision  recall  f1-score  support
0              0.571429  0.4000  0.470588  10.0000
1              0.333333  0.5000  0.400000   6.0000
accuracy       0.437500  0.4375  0.437500   0.4375
macro avg      0.452381  0.4500  0.435294  16.0000
weighted avg   0.482143  0.4375  0.444118  16.0000

Begin 

,Dataset Names,SVM GHG RATE F1-Score
0,Voter Data,0.4375
1,General Population Data,0.3125
2,Municipal Data,0.3750
3,Age Data,0.6250
4,Race/Ethnicity Data,0.5000
5,Family Unit Data,0.4375
6,Housing Unit Data,0.3125
7,Housing Affordability Data,0.4375
8,Household Income Data,0.3750
9,Economic Data,0.1875


In [31]:
alldatasets_SVC_landclass.to_csv('SVMLandOutput.csv')
alldatasets_SVC_rateclass.to_csv('SVMRateOutput.csv')

In [32]:
#Stochastic Gradient Descent
#Reference: https://stackoverflow.com/questions/39662398/scikit-learn-output-metrics-classification-report-into-csv-tab-delimited-format
from sklearn.linear_model import SGDClassifier 

def runSGD(dataframe, target_col):

    feature_names = dataframe.columns
    X_train, X_test, y_train, y_test = train_test_split(dataframe, target_col, random_state=42)
    X_scaler = StandardScaler().fit(X_train)
    X_train_scaled = X_scaler.transform(X_train)
    X_test_scaled = X_scaler.transform(X_test)

    
    sgd = SGDClassifier()
    sgd.fit(X_train_scaled, y_train)
    sgd_predictions = sgd.predict(X_test_scaled)
    output = sgd.score(X_test_scaled, y_test)
#     output = pd.DataFrame(classification_report(y_test, sgd_predictions, target_names = ["0", "1"], output_dict=True)).transpose()

    return output


In [33]:
#Naive Bayes Bernoulli Dist.
#Reference: https://stackoverflow.com/questions/39662398/scikit-learn-output-metrics-classification-report-into-csv-tab-delimited-format
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import CategoricalNB


def runGNB(dataframe, target_col):

    feature_names = dataframe.columns
    X_train, X_test, y_train, y_test = train_test_split(dataframe, target_col, random_state=42)
    X_scaler = StandardScaler().fit(X_train)
    X_train_scaled = X_scaler.transform(X_train)
    X_test_scaled = X_scaler.transform(X_test)

    
    gnb = GaussianNB()
    gnb_predictions = gnb.fit(X_train_scaled, y_train).predict(X_test_scaled)
    output = gnb.score(X_test_scaled, y_test)
#     output = pd.DataFrame(classification_report(y_test, sgd_predictions, target_names = ["0", "1"], output_dict=True)).transpose()

    return output


In [34]:
#Ridge Classification
#Reference: https://stackoverflow.com/questions/39662398/scikit-learn-output-metrics-classification-report-into-csv-tab-delimited-format
from sklearn.linear_model import RidgeClassifier

def runRidge(dataframe, target_col):

    feature_names = dataframe.columns
    X_train, X_test, y_train, y_test = train_test_split(dataframe, target_col, random_state=42)
    X_scaler = StandardScaler().fit(X_train)
    X_train_scaled = X_scaler.transform(X_train)
    X_test_scaled = X_scaler.transform(X_test)

    
    ridge = RidgeClassifier()
    ridge_predictions = ridge.fit(X_train_scaled, y_train).predict(X_test_scaled)
    output = ridge.score(X_test_scaled, y_test)
#     output = pd.DataFrame(classification_report(y_test, sgd_predictions, target_names = ["0", "1"], output_dict=True)).transpose()

    return output


In [35]:
def iterator(alldatasets, total_iterations, target_col, model_type):
    
    if model_type == 'sgd':
    
        for iteration in range(total_iterations):

            model_accuracies = [] 
            dataset_names = []

            for i in range(len(alldatasets)):

                model_output = runSGD(alldatasets[i]['data'], target_col)

    #             print("\nBegin Calculation for " + alldatasets[i]['name'])
    #             print('Overall Model Accuracy: ', rfm_output1['Accuracy'])
    #             print('Overall Importances: ')
    #             for importance in rfm_output1['Importances']:
    #                 print(importance)
                model_accuracies.append(model_output)
                dataset_names.append(alldatasets[i]['name'])
    #             print('End Calculation\n')

            if(iteration == 0):

                accuracies = pd.DataFrame({"Dataset Names": dataset_names, f"{target_col.name} Accuracy 0": model_accuracies})

            accuracies[f"{target_col.name} Accuracy " + str(iteration)] = model_accuracies


        print('End Calculation\n') 
        
        average = f"{model_type.upper()} {target_col.name} Average Accuracy"
        maximum = f"{model_type.upper()} {target_col.name} Max"
        minimum = f"{model_type.upper()} {target_col.name} Min"
        std_dev = f"{model_type.upper()} {target_col.name} Std"
        variance = f"{model_type.upper()} {target_col.name} Var"
        
        accuracies[average] = accuracies.mean('columns')
        accuracies[maximum] = accuracies.max('columns')
        accuracies[minimum] = accuracies.min('columns')
        accuracies[std_dev] = accuracies.std('columns')
        accuracies[variance] = accuracies.var('columns')

        accuracies = accuracies[['Dataset Names', average]]

        return accuracies
    
    if model_type == 'gnb':

        for iteration in range(total_iterations):

            model_accuracies = [] 
            dataset_names = []

            for i in range(len(alldatasets)):

                model_output = runGNB(alldatasets[i]['data'], target_col)

    #             print("\nBegin Calculation for " + alldatasets[i]['name'])
    #             print('Overall Model Accuracy: ', rfm_output1['Accuracy'])
    #             print('Overall Importances: ')
    #             for importance in rfm_output1['Importances']:
    #                 print(importance)
                model_accuracies.append(model_output)
                dataset_names.append(alldatasets[i]['name'])
    #             print('End Calculation\n')

            if(iteration == 0):

                accuracies = pd.DataFrame({"Dataset Names": dataset_names, f"{model_type.upper()} {target_col.name} Accuracy": model_accuracies})


        print('End Calculation\n') 

        return accuracies
    
    if model_type == 'ridge':

        for iteration in range(total_iterations):

            model_accuracies = [] 
            dataset_names = []

            for i in range(len(alldatasets)):

                model_output = runGNB(alldatasets[i]['data'], target_col)

    #             print("\nBegin Calculation for " + alldatasets[i]['name'])
    #             print('Overall Model Accuracy: ', rfm_output1['Accuracy'])
    #             print('Overall Importances: ')
    #             for importance in rfm_output1['Importances']:
    #                 print(importance)
                model_accuracies.append(model_output)
                dataset_names.append(alldatasets[i]['name'])
    #             print('End Calculation\n')

            if(iteration == 0):

                accuracies = pd.DataFrame({"Dataset Names": dataset_names, f"{model_type.upper()} {target_col.name} Accuracy": model_accuracies})


        print('End Calculation\n') 

        return accuracies
    

In [36]:
sgd_land = iterator(total_dfs, 50, target1, 'sgd')
sgd_rate = iterator(total_dfs, 50, target2, 'sgd')

sgd_land

End Calculation

End Calculation



,Dataset Names,SGD Land Class Average Accuracy
0,Voter Data,0.56875
1,General Population Data,0.59250
2,Municipal Data,0.61375
3,Age Data,0.56750
4,Race/Ethnicity Data,0.66250
5,Family Unit Data,0.54000
6,Housing Unit Data,0.72375
7,Housing Affordability Data,0.58750
8,Household Income Data,0.60875
9,Economic Data,0.56625


In [37]:
gnb_land = iterator(total_dfs, 1, target1, 'gnb')
gnb_rate = iterator(total_dfs, 1, target2, 'gnb')
gnb_land

End Calculation

End Calculation



,Dataset Names,GNB Land Class Accuracy
0,Voter Data,0.6875
1,General Population Data,0.6250
2,Municipal Data,0.6250
3,Age Data,0.6250
4,Race/Ethnicity Data,0.5625
5,Family Unit Data,0.5000
6,Housing Unit Data,0.6875
7,Housing Affordability Data,0.5625
8,Household Income Data,0.5625
9,Economic Data,0.5625


In [38]:
ridge_land = iterator(total_dfs, 1, target1, 'ridge')
ridge_rate = iterator(total_dfs, 1, target2, 'ridge')
ridge_land

End Calculation

End Calculation



,Dataset Names,RIDGE Land Class Accuracy
0,Voter Data,0.6875
1,General Population Data,0.6250
2,Municipal Data,0.6250
3,Age Data,0.6250
4,Race/Ethnicity Data,0.5625
5,Family Unit Data,0.5000
6,Housing Unit Data,0.6875
7,Housing Affordability Data,0.5625
8,Household Income Data,0.5625
9,Economic Data,0.5625


In [39]:
model_list = [sgd_land, sgd_rate, gnb_land, gnb_rate, ridge_land, ridge_rate, alldatasets_SVC_landclass, alldatasets_SVC_rateclass, avg_accuracy_df]

primary_df = model_list[0]

for dataframe in model_list[1:]:
    primary_df = primary_df.merge(dataframe, on='Dataset Names')
    
primary_df

,Dataset Names,SGD Land Class Average Accuracy,SGD GHG Rate Average Accuracy,GNB Land Class Accuracy,GNB GHG Rate Accuracy,RIDGE Land Class Accuracy,RIDGE GHG Rate Accuracy,SVM LAND CLASS F1-Score,SVM GHG RATE F1-Score,Land Classifier Avg. Accuracy,CO2 Rate Avg. Accuracy
0,Voter Data,0.56875,0.48125,0.6875,0.5000,0.6875,0.5000,0.5625,0.4375,0.50000,0.50125
1,General Population Data,0.59250,0.47375,0.6250,0.3750,0.6250,0.3750,0.4375,0.3125,0.64250,0.48250
2,Municipal Data,0.61375,0.48500,0.6250,0.4375,0.6250,0.4375,0.6875,0.3750,0.68625,0.49250
3,Age Data,0.56750,0.55500,0.6250,0.3125,0.6250,0.3125,0.8125,0.6250,0.70000,0.58625
4,Race/Ethnicity Data,0.66250,0.46125,0.5625,0.3750,0.5625,0.3750,0.5625,0.5000,0.71625,0.45750
5,Family Unit Data,0.54000,0.50875,0.5000,0.3750,0.5000,0.3750,0.7500,0.4375,0.65875,0.64625
6,Housing Unit Data,0.72375,0.46000,0.6875,0.4375,0.6875,0.4375,0.5625,0.3125,0.66125,0.44125
7,Housing Affordability Data,0.58750,0.50750,0.5625,0.5000,0.5625,0.5000,0.7500,0.4375,0.73375,0.56625
8,Household Income Data,0.60875,0.47875,0.5625,0.5625,0.5625,0.5625,0.6875,0.3750,0.59375,0.43125
9,Economic Data,0.56625,0.45625,0.5625,0.5000,0.5625,0.5000,0.8125,0.1875,0.66625,0.43375


In [41]:
primary_df.to_csv('all_classifier_analyses.csv')